In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import numpy as np

# Import API key
from api_keys import geoapify_key

In [16]:
# Get CSV from WeatherPy Folder
weather_data_to_load = "../WeatherPy/weather_df.csv"
weather_data = pd.read_csv(weather_data_to_load)

# Get rid of any empty data
update_weather_data = weather_data.dropna()
del update_weather_data["Unnamed: 0"]
update_weather_data.head(10)

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,faanui,PF,2023-02-03 22:34:54,-16.4833,-151.7500,14.0,74.0,82.166,7.71
1,dikson,RU,2023-02-03 22:35:46,73.5069,80.5464,100.0,100.0,-8.590,0.78
2,chanika,TZ,2023-02-03 22:35:46,-5.4167,38.0167,100.0,92.0,70.790,0.77
3,ushuaia,AR,2023-02-03 22:34:42,-54.8000,-68.3000,0.0,71.0,48.128,7.20
4,hirara,JP,2023-02-03 22:35:46,24.8000,125.2833,75.0,73.0,68.540,5.14
5,norman wells,CA,2023-02-03 22:35:47,65.2820,-126.8329,100.0,69.0,-23.494,2.06
6,mahebourg,MU,2023-02-03 22:35:47,-20.4081,57.7000,40.0,83.0,82.526,3.09
7,albany,US,2023-02-03 22:30:01,42.6001,-73.9662,80.0,45.0,-6.376,2.68
8,cape town,ZA,2023-02-03 22:31:42,-33.9258,18.4232,0.0,75.0,68.648,2.06
10,saint-philippe,RE,2023-02-03 22:35:48,-21.3585,55.7679,56.0,86.0,80.456,4.76


In [17]:
#  
map_plot1=update_weather_data.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True,
    tiles='OSM'
)
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]

In [24]:
# Add humidity
humidity=update_weather_data['Humidity (%)'].astype(float)

hum_plot=update_weather_data.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True,
    size='Humidity (%)',
    scale= 1,
    tiles='OSM',
    color='City'
)
hum_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

In [19]:
# Narrow down options

# Filter vacation with zero cloudiness
vacation_clear = update_weather_data[update_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 75F
vacation_over75 = vacation_clear[vacation_clear["Max Temp (F)"] > 75]
# Filter vacation with max temp below 85F
vacation_under85 = vacation_over75[vacation_over75["Max Temp (F)"] < 85]
# Filter vacation with wind speed below 7.5mph
vacation_wind = vacation_under85[vacation_under85["Wind Speed (mph)"] < 7.5]
# Filter vacation with humidity below 70%
ideal_vacation = vacation_wind[vacation_wind["Humidity (%)"] < 70]
# Set Index
updated_perfect_vacation = ideal_vacation.reset_index()
del updated_perfect_vacation["index"]
updated_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,yulara,AU,2023-02-03 22:35:57,-25.2406,130.9889,0.0,21.0,83.534,4.84
1,comodoro rivadavia,AR,2023-02-03 22:36:02,-45.8667,-67.5000,0.0,22.0,80.726,4.63
2,flinders,AU,2023-02-03 22:36:46,-34.5833,150.8552,0.0,38.0,80.456,5.36
3,port macquarie,AU,2023-02-03 22:35:30,-31.4333,152.9167,0.0,34.0,81.158,5.54
4,alice springs,AU,2023-02-03 22:32:35,-23.7000,133.8833,0.0,34.0,78.620,5.66
5,neuquen,AR,2023-02-03 22:37:57,-38.9516,-68.0591,0.0,50.0,75.362,0.00
6,tezu,IN,2023-02-03 22:38:47,27.9167,96.1667,0.0,15.0,79.934,1.79
7,along,IN,2023-02-03 22:38:53,28.1667,94.7667,0.0,25.0,76.946,0.71


In [30]:
hotels_df=updated_perfect_vacation.filter(['City', 'Country', 'Latitude', 'Longitude','Humidity (%)'],axis=1)
hotels_df['Hotel']=''
hotels_df

,City,Country,Latitude,Longitude,Humidity (%),Hotel
0,yulara,AU,-25.2406,130.9889,21.0,
1,comodoro rivadavia,AR,-45.8667,-67.5000,22.0,
2,flinders,AU,-34.5833,150.8552,38.0,
3,port macquarie,AU,-31.4333,152.9167,34.0,
4,alice springs,AU,-23.7000,133.8833,34.0,
5,neuquen,AR,-38.9516,-68.0591,50.0,
6,tezu,IN,27.9167,96.1667,15.0,
7,along,IN,28.1667,94.7667,25.0,


In [8]:
hotels_df.dtypes

City              object
Country           object
Latitude         float64
Longitude        float64
Nearest Hotel     object
dtype: object

In [31]:
# Set the parameters for the type of search
limit = 10
radius = 100000


# set up a parameters dictionary
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
   lat =row['Latitude']
   lon =row['Longitude']
   # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   params["filter"] = f"circle:{lon},{lat},{radius}"
   params["bias"] = f"proximity:{lon},{lat}"
   # Set base URL
   base_url = "https://api.geoapify.com/v2/places"
   # Make and API request using the params dictionary
   name_address =requests.get(base_url, params=params)
   # Convert the API response to JSON format
   name_address = name_address.json()
   # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   try:
        hotels_df.loc[index, "Hotel"] = name_address["features"][0]["properties"]["name"]
    # If no hotel is found, set the hotel name as "No hotel found".
   except (KeyError, IndexError):
        hotels_df.loc[index, "Hotel"] = "No hotel found"     


   
   

        

       
        


# Log the search results
print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel']}")

# Display sample data
hotels_df


Starting hotel search
along - nearest hotel: Hotel Toshi Palace


,City,Country,Latitude,Longitude,Humidity (%),Hotel
0,yulara,AU,-25.2406,130.9889,21.0,Outback Pioneer Hotel & Lodge
1,comodoro rivadavia,AR,-45.8667,-67.5000,22.0,No hotel found
2,flinders,AU,-34.5833,150.8552,38.0,Nova Kiama
3,port macquarie,AU,-31.4333,152.9167,34.0,The Observatory
4,alice springs,AU,-23.7000,133.8833,34.0,Aurora Alice Springs
5,neuquen,AR,-38.9516,-68.0591,50.0,Hotel Argentina 307
6,tezu,IN,27.9167,96.1667,15.0,Mimu
7,along,IN,28.1667,94.7667,25.0,Hotel Toshi Palace


In [32]:
# Add the hotel name and country as additional information for each city in the map
# Configure the map plot and Display the map
vac_plot=hotels_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo=True,
    size='Humidity (%)',
    scale= 1,
    tiles='OSM',
    color='City',
    hover_cols=['Hotel','Country']
    )
vac_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%),Hotel,Country)